In [1]:
import re
import pandas as pd
import numpy as np
import itertools
import os

In [2]:
path = 'E:\Research_Data\Vent_Dyssynchrony\Data\Epic_Data'

In [3]:
def data_analysis(fileName):
    raw_data = []

    data = open(fileName, 'r+')
    text = data.read()
    entries = re.split('\n\n', text)

    for items in entries:
            raw_data.append(items.strip('\n '))

    search_items = {'DateTime': {'head':'', 'pat':'(\d\d/\d\d/\d\d\n \d\d\d\d)|(\d\d/\d\d/\d\d\n \d\d:\d\d:\d\d)|(\d\d/\d\d/\d\d\n \d\d:\d\d:\d\d:\d\d)'},
                    'BP': {'head':'BP(?! )', 'pat': '\d\d\d?/\d\d?\d?'},
                    'MAP': {'head':'(?!= A-line )MAP', 'pat':'\d\d\d?'},
                    'Resp':{'head':'Resp(?![a-z ])', 'pat':'\d\d?'},
                    'Pulse': {'head':'Pulse', 'pat':'\d\d?\d?'},
                    'A-line': {'head':'A-line(?! MAP)', 'pat':'\d\d\d?/\d\d?\d?'},
                    'A-line MAP': {'head':'A-line MAP', 'pat':'\d\d\d?'},
                    'CVP':{'head':'CVP ', 'pat': '\d\d?'},
                    'SpO2': {'head':'(?!=-)SpO2(?!-)', 'pat':'\d\d+'},
                    'FiO2': {'head':'FiO2', 'pat':'(\d\d+)|(\d\d\d)'},
                    'Vent Mode': {'head':'Vent Mode', 'pat':'(PRVC)|((?<!;)CPAP)|(PSV(?!;))|(PSV;CPAP)|(APV(?!;))|((?<!;)CMV)|(APV;CMV)'},
                    'Set Vt': {'head':'Set Vt', 'pat':'\d\d\d'},
                    'Set RR': {'head':'Set RR', 'pat':'\d\d?'},
                    'Position': {'head':'Repositioned', 'pat':'\w+'},
                    'RASS': {'head':'RASS Sedation Scale', 'pat':'.\d'},
                    'TOF': {'head':'Twitches', 'pat':'\d' },
                    'PEEP':{'head':'PEEP', 'pat': '\d\d?'},
                    'Plat':{'head':'Static Pressure', 'pat':'\d+'},
                    'iNO':{'head':'NO(?!2)', 'pat': '(\d\d)|(0\.\d)'},
                    'CPOT Tot':{'head': 'CPOT Total', 'pat':'\d+'},
                    'CPOT Vent':{'head':'Ventilator Compliance', 'pat':'\d+'}}


    found_items = {'DateTime': {'rows':0, 'values':[]},
                    'BP': {'rows':0, 'values':[]},
                    'MAP': {'rows':0, 'values':[]},
                    'Pulse': {'rows':0, 'values':[]},
                    'Resp':{'rows':0, 'values':[]},
                    'A-line': {'rows':0, 'values':[]},
                    'A-line MAP': {'rows':0, 'values':[]},
                    'CVP': {'rows':0, 'values':[]},
                    'SpO2': {'rows':0, 'values':[]},
                    'FiO2': {'rows':0, 'values':[]},
                    'Vent Mode': {'rows':0, 'values':[]},
                    'Set Vt': {'rows':0, 'values':[]},
                    'Set RR': {'rows':0, 'values':[]},
                    'Position': {'rows':0, 'values':[]},
                    'RASS': {'rows':0, 'values':[]},
                    'TOF': {'rows':0, 'values':[]},
                    'PEEP': {'rows':0, 'values':[]},
                    'Plat': {'rows':0, 'values':[]},
                    'iNO': {'rows':0, 'values':[]},
                    'CPOT Tot':{'rows':0, 'values':[]},
                    'CPOT Vent':{'rows':0, 'values':[]},}

    final_data = {'DateTime':['BP', 'MAP', 'Pulse', 'A-line', 'A-line MAP', 'CVP','SpO2', 'FiO2', 'Resp', 'Vent Mode', 'Set Vt', 'Set RR', 'Position', 
                              'RASS', 'TOF', 'PEEP', 'Plat', 'iNO', 'CPOT Tot', 'CPOT Vent']}

    def date_cleaner(text):
        clean_up = text.split('\n')
        text = clean_up[0] + clean_up[1]

        if ':' in text:
            clean_up = text.split(':')
            text = clean_up[0] + clean_up[1]
        return text

    count = 0
    for items in raw_data:
        if items == 'Vitals':
            count = count + 1

    print (count)

    for items in raw_data:
        results = re.match(search_items['DateTime']['pat'], items)

        if results:
            text = results.group(0)
            text = date_cleaner(text)
            found_items['DateTime']['rows'] = found_items['DateTime']['rows'] + 1
            found_items['DateTime']['values'].append(text)

    for items in search_items:
        if items != 'DateTime':
            for i in range(0, len(raw_data)):
                #Match to Desired Heading
                results = re.match(search_items[items]['head'], raw_data[i])

                if results:
                    #if positive patch, set up blank variables
                    temp = []            #obtain surrounding data
                    no_date = True
                    k=i
                    dates = []

                    #update number of hits for that heading
                    found_items[items]['rows'] = found_items[items]['rows'] + 1

                    #match desired data and blanks to keep spacing in temp list
                    for k in range(i, i+19):                   
                        sub_results = re.match(search_items[items]['pat'], raw_data[k])

                        if sub_results:
                            temp.append(sub_results.group(0))
                        else:
                            temp.append(raw_data[k])

                    #special management of SPO2, Resp, HR because of Epic
                    to_pop=[]
                    if items == 'SpO2':
                        for y in range(0, len(temp)-1):
                            if temp[y].isdecimal() and int(temp[y])< 90:
                                to_pop.append(y-1)
                        for values in to_pop[::-1]:
                            temp.pop(values)
                    elif items == 'Resp':
                        for y in range(0, len(temp)-1):
                            if temp[y].isdecimal() and int(temp[y])> 30:
                                to_pop.append(y-1)
                        for values in to_pop[::-1]:
                            temp.pop(values)
                    elif items == 'Pulse':
                        for y in range(0, len(temp)-1):
                            if temp[y].isdecimal() and int(temp[y])< 60:
                                to_pop.append(y-1)
                        for values in to_pop[::-1]:
                            temp.pop(values)
                            

                    #find most recent set of date data to match with
                    while no_date:
                        date_results = re.match(search_items['DateTime']['pat'], raw_data[k])

                        if date_results:
                            for j in range (k, k-12, -1):
                                more_dates = re.match(search_items['DateTime']['pat'], raw_data[j])
                                if more_dates:
                                    dates.append(date_cleaner(more_dates.group(0)))
                            no_date=False

                        k = k-1

                    #print('\t', dates[::-1], '\n', temp)

                    #zip dates and values into tuple
                    zipped = zip(dates[::-1], temp[1:])   
                    found_items[items]['values'].append(list(zipped))

    for items in found_items:
        if items != 'DateTime':
            lists = list(itertools.chain(*found_items[items]['values']))
            found_items[items]['values'] = lists

    for items in found_items:
        print (items, len(found_items[items]['values']), found_items[items]['rows']*5)
        #print (found_items[items]['values'])

    dicts = []
    for items in final_data['DateTime']:
        for values in found_items[items]['values']:
            dicts.append({items:values[1], 'DateTime':values[0]})

    df = pd.DataFrame.from_dict(found_items['DateTime']['values'])
    df.rename(columns={0:'DateTime'}, inplace=True)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df.set_index('DateTime', inplace=True, drop=True, verify_integrity=True)

    temp_df = pd.DataFrame(dicts)
    temp_df.replace(to_replace='', value=np.nan, inplace=True)
    temp_df.replace(to_replace='--', value=np.nan, inplace=True)
    temp_df['DateTime'] = pd.to_datetime(temp_df['DateTime'])
    temp_df.set_index('DateTime', inplace=True, drop=True)

    temp_df.dropna(inplace=True, axis=0, how='all')

    cols = temp_df.columns
    for col in cols:
        temp = temp_df[col]
        temp.dropna(inplace=True)
        temp
        df = df.join(temp, how='left')

    df.dropna(inplace=True, how='all')
    print(df.count())
    
    return df

In [4]:
def lab_analysis(fileName):
    class labTypes:
        def __init__ (self, dateTime, labName, labValue):
            self.dateTime = dateTime
            self.labName = labName
            self.labValue = labValue
        
    file = open(path+'\\'+ fileName)
    fileLines = file.readlines()
    file.close()
    
    labCollection = []
    labNames = []
    dates = []
    title = "Date/Times,"
    
    for lines in fileLines:
        if re.match(r'\d\d?/\d\d?/\d\d\d\d \d\d:\d\d', lines):
            dateTime = lines.strip()
            
        if re.search(r'\w: \d', lines):
            lineInfo = lines.split(": ")
            labName = lineInfo[0]
            
            temp = lineInfo[1].split(" (")
            labValue = temp[0]
    
            temp = labName.split(",")
            labName = ""
            labName = labName.join(temp)
            labValue = labValue.strip()
    
            labCollection.append(labTypes(dateTime, labName, labValue))
    
            if labName not in labNames:
                labNames.append(labName)
    
            if dateTime not in dates:
                dates.append(dateTime)
    
    dates.sort()
    labNames.sort()
    
    
    for labs in labNames:
        title = title + labs + ", "
    
    for number in dates:
        title = title + "\n" + number + ","
    
        for items in labNames:
            addedItem = 0
    
            for things in labCollection:
                if number == things.dateTime and items == things.labName:
                    addedItem = 1
                    title = title + things.labValue + ","
    
            if addedItem == 0:
                title = title + "--,"
    
    return (title)

In [5]:
directory = os.listdir(path)
data_files = []
all_files = []

for patients in directory:
    file_list = os.listdir(path + '\\' + patients)

    for files in file_list:
        all_files.append([path, patients, files])
        if '.txt' in files and os.path.getsize(path +'\\'+ patients +'\\'+ files) > 0:
                if ('RN' in files or 'RT' in files) and 'edit' not in files:                          
                    if os.path.exists(path +'\\'+ patients +'\\edited'+ files) == False:
                        try:
                            df = data_analysis(path +'\\'+ patients +'\\'+ files)
                            print(patients, files)
                            print('\n')
                            df.to_csv(path +'\\'+ patients +'\\edited'+ files)
                        except Exception as e:
                            print('did not save', patients, files, e, '\n\n\n')
                if ('Lab' in files) and 'edit' not in files:
                        df = lab_analysis('\\' + patients +'\\'+ files)
                        newFile = open(path+ '\\' +patients + '\\edit'+ files, "w")
                        newFile.write(df)
                        newFile.close()

148
DateTime 738 3690
iNO 0 0
Vent Mode 428 430
Resp 693 695
Pulse 743 745
CVP 365 365
Position 638 640
RASS 543 545
SpO2 738 740
TOF 160 160
A-line 433 435
CPOT Tot 260 260
FiO2 433 435
MAP 333 335
PEEP 433 435
CPOT Vent 260 260
A-line MAP 433 435
BP 363 365
Plat 0 0
Set Vt 193 195
Set RR 353 355
did not save P122 RN Data.txt Index has duplicate keys: <class 'pandas.tseries.index.DatetimeIndex'>
[2015-04-14 11:28:00]
Length: 1, Freq: None, Timezone: None 



0
DateTime 697 3485
iNO 0 0
Vent Mode 431 435
Resp 652 660
Pulse 702 710
CVP 370 370
Position 150 150
RASS 0 0
SpO2 697 705
TOF 0 0
A-line 437 445
CPOT Tot 0 0
FiO2 441 445
MAP 0 0
PEEP 436 440
CPOT Vent 0 0
A-line MAP 0 0
BP 326 330
Plat 90 90
Set Vt 185 185
Set RR 361 365
did not save P122 RT Data.txt Index has duplicate keys: <class 'pandas.tseries.index.DatetimeIndex'>
[2015-04-14 11:28:00]
Length: 1, Freq: None, Timezone: None 





AttributeError: 'str' object has no attribute 'head'